In [4]:
import pandas as pd

In [5]:
#import files
tmc_r = pd.read_excel('TMC_Report.xlsx')
os_r = pd.read_excel('OS_Report.xls')
tmc_id = pd.read_excel('TMC_ID.xlsx')
os_id = pd.read_excel('OS_ID.xlsx')

In [7]:
os_r['Origin'] = os_r['Origin'].str.strip()

#Oversight - count trips by origin location
os_origin_trip_count = os_r.groupby('Origin').agg('Global Device ID').count()
os_origin_trip_count= pd.DataFrame(os_origin_trip_count).sort_values(by='Global Device ID', ascending=False)

#Merge Oversight report with Oversight ID locations
os_table_id_trips = pd.merge(os_origin_trip_count, 
                      os_id, 
                      on ='Origin', 
                      how ='outer')


In [8]:
tmc_r = tmc_r[tmc_r['Equipment Type'] == 'Reefer']

In [10]:
#TMC - count trips by location ID
tmc_origin_trip_count = tmc_r.groupby('Origin Pickup Customer Location ID').agg('CHR Number').count()
tmc_origin_trip_count= pd.DataFrame(tmc_origin_trip_count).sort_values(by='CHR Number', ascending=False)
tmc_origin_trip_count.reset_index(inplace=True)

#Merge TMC report with TMC ID locations
tmc_table_id_trips = pd.merge(tmc_origin_trip_count, 
                      tmc_id, 
                      on ='Origin Pickup Customer Location ID', 
                      how ='outer')

In [11]:
#Columns rename for merging
tmc_table_id_trips = tmc_table_id_trips.rename(columns={'Origin Pickup Customer Location ID': 'Origin_ID'})
os_table_id_trips = os_table_id_trips.rename(columns={'OS ID map': 'Origin_ID'})

In [12]:
#Merge oversigt and TMC data based on Origin_ID (TMC table)
master = pd.merge(tmc_table_id_trips, 
                      os_table_id_trips, 
                      on ='Origin_ID', 
                      how ='outer')
master.drop_duplicates(subset=['Origin_ID'], inplace=True)

In [14]:
#Add compliance calculation - # of OS trips / # of TMC trips
master['Compliance Shipments'] = master['Global Device ID'] / master['CHR Number']
master.set_index('Origin_ID', inplace=True)

In [15]:
#master.to_excel('report_by_ids.xlsx')

# Groupby by names

In [16]:
#Example # 2

merge_names = master

In [17]:
merge_names.reset_index(inplace=True)
add_names = merge_names.groupby('Supplier').agg({'CHR Number':'sum', 
                         'Global Device ID':'sum'})
add_names.reset_index(inplace=True)


In [18]:
final = add_names.sort_values(by='CHR Number', ascending=False)
final['Compliance'] = final['Global Device ID'] / final['CHR Number']
final.to_excel('report_by_name.xlsx')

In [19]:
add_names = merge_names.groupby(['Origin','Supplier']).agg({'CHR Number':'sum', 
                         'Global Device ID':'sum'})
add_names.reset_index(inplace=True)

# List of origins w/ out IDs and new origins

In [20]:
list_origin = os_r['Origin']
#list_origin = os_r.drop_duplicates

In [21]:
df = pd.DataFrame(list_origin.drop_duplicates())

In [22]:
df['source'] = 'Monthly Report'

In [23]:
os_id['Source 2'] = 'Id Report'

In [24]:
origin_missing = pd.merge(os_id, 
                      df, 
                      on ='Origin', 
                      how ='outer')

In [25]:
origin_missing.drop_duplicates(inplace=True)

In [26]:
origin_not_listed = origin_missing[(origin_missing['OS ID map'].isnull())]

In [27]:
origin_not_listed.drop(['source', 'Source 2'], axis=1, inplace= True)
os_id.drop(['Source 2'], axis=1, inplace=True)

C:\Users\ricardo.lugo\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
#origin_not_listed.to_excel('origin_to_be_added.xlsx')

In [29]:
os_id = os_id.append(origin_not_listed, ignore_index=True)
os_id.drop_duplicates(subset=['Origin'], inplace=True)

In [30]:
os_id.set_index('Origin', inplace=True)

In [31]:
os_id.to_excel('OS_ID.xlsx')